<a href="https://colab.research.google.com/github/DannMacias/customer-segmentation/blob/main/01_data_cleaning_feature_engineering_customer_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b> Customer Segmentation 🛒: Data cleaning and feature engineering </b>

**What is customer segmentation?**

Customer segmentation consist of grouping customers together based on similar features or properties.

**Why segment customers?**
* Improving your customer service and customer support efforts.
* Helping internal teams prepare for challenges different groups are likely to experience.
* Communicating with segments of customers through prederred channels or platforms.
* Finding new opportunities for products, support, and service efficiently.

**Goal**

Perform clustering to summarize customer segments. The first thing we need to do is a data cleaning and some feature engineering.

# 0. Getting setup

In [1]:
# Import basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

The data is for [Kaggle Customer Personality Analysis](https://www.kaggle.com/datasets/imakash3011/customer-personality-analysis).

In [2]:
# Get the data from GitHub repo
data_path = "https://github.com/DannMacias/customer-segmentation/raw/main/customer-analysis-database.csv"
df = pd.read_csv(data_path, sep = "\t")
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


In [3]:
# Number of data points
print(f"Number of datapoints: {len(df)}")

Number of datapoints: 2240


In [4]:
# Get the columns names
df.columns

Index(['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response'],
      dtype='object')

# 1. Data Cleaning and feature engineering

In [5]:
# Information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

In [6]:
# How many missing values (NaN) we get?
df.isna().sum()

ID                      0
Year_Birth              0
Education               0
Marital_Status          0
Income                 24
Kidhome                 0
Teenhome                0
Dt_Customer             0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
AcceptedCmp3            0
AcceptedCmp4            0
AcceptedCmp5            0
AcceptedCmp1            0
AcceptedCmp2            0
Complain                0
Z_CostContact           0
Z_Revenue               0
Response                0
dtype: int64

* `Dt_customer`, that indicates the date a customer joined the database, is not parsed as DateTime.
* There are missing values in `Income`.
* `Education` and `Marital_status` are categorical features, maybe encode that features later on.

For all the missing values, I'll drop the rows that have missing income values.

In [7]:
# Remove missing values
df = df.dropna()
print(f"Total number of data poitns after removing the rows with missing values: {len(df)}")

Total number of data poitns after removing the rows with missing values: 2216


I'll create a new feature called `Total_Spent` indicating the total amount spent by the customers in various categorias over the span of two years.

In [8]:
# Create feature Total_Spent
df["Total_Spent"] = df["MntWines"] + df["MntFruits"] + df["MntMeatProducts"] + df["MntFishProducts"] + df["MntSweetProducts"] + df["MntGoldProds"]
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,Total_Spent
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,0,0,0,0,0,0,3,11,1,1617
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,0,0,0,0,0,0,3,11,0,27
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,0,0,0,0,0,0,3,11,0,776
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,0,0,0,0,0,0,3,11,0,53
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,0,0,0,0,0,0,3,11,0,422


Create another features:
* `Age` - convert the `Year_Birth` to an age feature.
  * *Note*: the dataset is from 2014, this prevents outliers with an age of 130 years.
* `Seniority` - indicates the number of days the customers started to shop in the store based on the laste recorded date.
* `Partner` - indicates the living situation of couples, based on `Marital_Status`.
* `Num_Children` - indictes the total children in the households, that is kids and teenagers.
* `Family_Size` - indactes the number of persons living in the household.
* `Is_Parent` - indicate parenthood status.
* Simplify the `Education` feature.

In [9]:
# Create Age feature
df["Age"] = 2014 - df["Year_Birth"]
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,Total_Spent,Age
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,0,0,0,0,0,3,11,1,1617,57
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,0,0,0,0,0,3,11,0,27,60
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,0,0,0,0,0,3,11,0,776,49
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,0,0,0,0,0,3,11,0,53,30
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,0,0,0,0,0,3,11,0,422,33


In [ ]:
# Convert Dt_Customer to a datetime feature
df["Dt_Customer"] = pd.to_datetime(df["Dt_Customer"])

# Get the last enrolment date in the record
dates = []
for date in df["Dt_Customer"]:
  date = date.date()
  dates.append(date)

In [11]:
# Get the oldest and the newest customer's date
print(f"The oldest customer's enrolment date in the records: {min(dates)}")
print(f"The newst customer's enrolment date in the records: {max(dates)}")

The oldest customer's enrolment date in the records: 2012-01-08
The newst customer's enrolment date in the records: 2014-12-06


In [12]:
# Create Seniority feature
df["Seniority"] = pd.to_datetime(df["Dt_Customer"], dayfirst = True, format = "%Y-%m-%d")

# Get the last day
last_date = max(dates)

df["Seniority"] = df["Seniority"].dt.date.apply(lambda x: (last_date - x)).dt.days

# Convert seniority into a numeric value
df["Seniority"] = pd.to_numeric(df["Seniority"], downcast = "integer")

In [13]:
# Partner feature
partner_dict = {"Married": "Partner",
                "Together": "Partner",
                "Absurd": "Alone",
                "Widow": "Alone",
                "YOLO": "Alone",
                "Divorced": "Alone",
                "Single": "Alone"}

df["Partner"] = df["Marital_Status"].replace(partner_dict)

In [14]:
# Num_Children feature
df["Num_Children"] = df["Kidhome"] + df["Teenhome"]

In [15]:
# Family_Size feature
df["Family_Size"] = df["Partner"].replace({"Alone": 1, "Partner": 2}) + df["Num_Children"]

In [16]:
# Is parent feature
df["Is_Parent"] = np.where(df["Num_Children"] > 0, 1, 0)

In [17]:
# Change Education feature values
education_dict = {"Graduation": "Graduate",
                  "PhD": "Postgraduate",
                  "Master": "Postgraduate",
                  "Basic": "Undergraduate",
                  "2n Cycle": "Undergraduate"}

df["Education"] = df["Education"].replace(education_dict)

In [18]:
# Change the number of sell products for clarity
products_dict = {"MntWines": "Wines",
                 "MntFruits": "Fruits",
                 "MntMeatProducts": "Meat",
                 "MntFishProducts": "Fish",
                 "MntSweetProducts": "Sweets",
                 "MntGoldProds": "Gold"}

df = df.rename(columns = products_dict)

In [19]:
# Dropping redundant features
features_to_drop = ["Marital_Status", "Dt_Customer", "Z_CostContact", "Z_Revenue", "Year_Birth", "ID"]
df = df.drop(features_to_drop, axis = 1)

In [20]:
df.head()

,Education,Income,Kidhome,Teenhome,Recency,Wines,Fruits,Meat,Fish,Sweets,...,AcceptedCmp2,Complain,Response,Total_Spent,Age,Seniority,Partner,Num_Children,Family_Size,Is_Parent
0,Graduate,58138.0,0,0,58,635,88,546,172,88,...,0,0,1,1617,57,971,Alone,0,1,0
1,Graduate,46344.0,1,1,38,11,1,6,2,1,...,0,0,0,27,60,125,Alone,2,3,1
2,Graduate,71613.0,0,0,26,426,49,127,111,21,...,0,0,0,776,49,472,Partner,0,2,0
3,Graduate,26646.0,1,0,26,11,4,20,10,3,...,0,0,0,53,30,65,Partner,1,3,1
4,Postgraduate,58293.0,1,0,94,173,43,118,46,27,...,0,0,0,422,33,321,Partner,1,3,1


In [21]:
#  Export the clean data as a csv
df.to_csv("clean_customer_analysis_database.csv", index = False)